In [1]:
import pandas as pd
import numpy as np

In [2]:
a = pd.read_csv("/Users/Nilavan/Documents/Jupyter/TM2/4NIL.csv", sep = ';',low_memory= False, decimal= ',').filter(['F2pt','F2_AGE']).dropna()

In [3]:
b = pd.read_csv("/Users/Nilavan/Documents/Jupyter/TM2/4NIL.csv", sep = ';', low_memory= False, decimal= ',').filter(['pt_','F2wt','F2hip','F2BMI']).dropna()

In [4]:
c = pd.read_csv("/Users/Nilavan/Documents/Jupyter/TM2/4NIL.csv", sep = ';', low_memory= False, decimal= ',').filter(['pt_1','F2HRTRTE']).dropna()

In [5]:
d = pd.read_csv("/Users/Nilavan/Documents/Jupyter/TM2/PA.csv", sep = ';', low_memory= False, decimal= ',').filter(['F2pt','F2nightnumber','F2dur_tmodday','F2dur_tvigday']).dropna()

In [6]:
dfm = pd.read_csv("/Users/Nilavan/Documents/Jupyter/TM2/MDDraw.csv", sep = ',', index_col = 0)

In [12]:
a = a.rename(columns = {'F2pt': 'pt'})
b = b.rename(columns = {'pt_': 'pt'})
c = c.rename(columns = {'pt_1': 'pt'})
d = d.rename(columns = {'F2pt': 'pt'})

In [42]:
d['F2dur_tmodday'] = d['F2dur_tmodday'].astype('float') #changer en float
d['F2dur_tvigday'] = d['F2dur_tvigday'].astype('float') #changer en float
d['MVPA'] = d['F2dur_tmodday'] + d['F2dur_tvigday'] #MVPA
d_ = d.groupby('pt').MVPA.mean()

In [55]:
ab = pd.merge(a,b, on = "pt")
abc = pd.merge(ab,c, on = "pt")
df = pd.merge(abc,d_, on = "pt") #somatic
df = df.drop(df.loc[df.MVPA==0].index)
df = df.rename(columns = {'pt': 'CoLausID'})
df['CoLausID'] = df['CoLausID'] + 50000
df_ = pd.merge(df,dfm, on = "CoLausID")
df_ = df_.drop(['F2_AGE_x'], axis =1)

In [54]:
d_.loc[6747]

94.77381007714287

In [59]:
df_['PAL'] = df_['MVPA']*7/30 # nombres de sessions de 30mn de MVPA par semaine
def PALs(x) : # PAL coefficient (Raichlen et al.)
        if x == 0 :
            return 0
        if 0< x <=1.5:
            return 0.39
        if 1.5< x <=3:
            return 0.35
        if 3< x <=6:
            return 0.64
        if x >6:
            return 1.21  
df_["PALc"]= df_.PAL.apply(lambda x : PALs(x))
df_['CRF'] = 3.5 * ((2.78 * df_['gender']) - (0.11 * df_['F2_AGE_y']) - (0.17 * df_['F2BMI']) - (0.05 * df_['F2HRTRTE']) + df_['PALc'] + 21.41)

#equation eCRF avec multiple de 3.5 : 1 MET = 3.5ml/O2..

In [61]:
df_.groupby(['gender','FULST_MDDPD2']).mean()

CoLausID       F2wt      F2BMI   F2HRTRTE       MVPA  \
gender FULST_MDDPD2                                                             
0      0             52722.128788  66.951010  25.727487  68.166667  25.745015   
       1             52894.177536  68.137862  25.809454  67.619565  25.825026   
1      0             52678.947368  81.852047  26.703124  66.623782  26.893142   
       1             52960.743421  83.079276  26.896117  66.018092  26.614386   

                      F2_AGE_y  F2_CL_age  FULST_MDDPDON  \
gender FULST_MDDPD2                                        
0      0             65.717139  65.108769            NaN   
       1             61.826587  61.170361      38.151852   
1      0             63.353888  62.746285            NaN   
       1             60.304886  59.680806      38.910331   

                     bsf2_lif_e_unsp_h_DSM_wt  bsf2_lif_e_mela_h_DSM_wt  \
gender FULST_MDDPD2                                                       
0      0                             0.000000                  0.000000   
       1                             0.386861                  0.246350   
1      0                             0.000000                  0.000000   
       1                             0.461538                  0.270903   

                     bsf2_lif_e_combi_h_DSM_wt  bsf2_lif_e_atyp_h_DSM_wt  \
gender FULST_MDDPD2                                                        
0      0                              0.000000                  0.000000   
       1                              0.213504                  0.153285   
1      0                              0.000000                  0.000000   
       1                              0.137124                  0.130435   

                      F2_GAF_C   F2_GAF_W   F2_GAF_L       PAL      PALc  \
gender FULST_MDDPD2                                                        
0      0             82.982278  77.108861  81.306329  6.007170  0.794520   
       1             76.003630  64.769510  73.039927  6.025839  0.802862   
1      0             84.212476  78.319688  82.372320  6.275066  0.811910   
       1             80.986842  71.026316  75.694079  6.210023  0.799474   

                           CRF  
gender FULST_MDDPD2             
0      0             25.177701  
       1             26.751733  
1      0             35.567919  
       1             36.689421

In [60]:
df_

,CoLausID,F2wt,F2BMI,F2HRTRTE,MVPA,gender,F2_AGE_y,F2_CL_age,FULST_MDDPD2,FULST_MDDPDON,bsf2_lif_e_unsp_h_DSM_wt,bsf2_lif_e_mela_h_DSM_wt,bsf2_lif_e_combi_h_DSM_wt,bsf2_lif_e_atyp_h_DSM_wt,F2_GAF_C,F2_GAF_W,F2_GAF_L,PAL,PALc,CRF
0,50002.0,99.9,33.301952,76.5,19.631944,1,69.6,68.0356,0,NaN,0.0,0.0,0.0,0.0,90.0,90.0,90.0,4.580787,0.64,26.906839
1,50004.0,59.4,21.298719,62.5,96.187500,0,52.8,52.1533,0,NaN,0.0,0.0,0.0,0.0,90.0,90.0,90.0,22.443750,1.21,35.231762
2,50005.0,54.3,20.188875,59.5,4.854167,0,59.0,58.4257,0,NaN,0.0,0.0,0.0,0.0,90.0,90.0,90.0,1.132639,0.39,31.160119
3,50007.0,79.3,34.098362,59.5,1.145833,0,81.7,81.3005,0,NaN,0.0,0.0,0.0,0.0,70.0,60.0,70.0,0.267361,0.39,14.144475
4,50009.0,73.3,26.345829,81.5,10.083333,0,59.2,58.6585,1,15.0,0.0,1.0,0.0,0.0,70.0,50.0,60.0,2.352778,0.35,23.429732
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1760,55455.0,74.0,31.819403,69.0,19.850000,0,50.6,49.7632,1,33.0,0.0,1.0,0.0,0.0,90.0,75.0,85.0,4.631667,0.64,26.686455
1761,55457.0,82.7,29.653269,67.5,75.371212,0,48.1,47.4415,1,25.0,0.0,0.0,1.0,0.0,65.0,60.0,70.0,17.586616,1.21,31.195305
1762,55461.0,88.2,26.193676,60.5,22.402778,1,55.6,54.2396,0,NaN,0.0,0.0,0.0,0.0,90.0,90.0,90.0,5.227315,0.64,39.326263
1763,55462.0,84.0,27.117769,66.5,32.319444,1,69.7,69.0951,0,NaN,0.0,0.0,0.0,0.0,90.0,80.0,85.0,7.541204,1.21,34.292927
